In [2]:
import pandas as pd
import numpy as np

In [3]:
station = pd.read_csv('data/station.csv', usecols=['id','city'])
station.head(3)

,id,city
0,2,San Jose
1,3,San Jose
2,4,San Jose


In [4]:
trip = pd.read_csv('data/trip_train.csv',
                   usecols=['duration', 'start_date', 'start_station_id', 'subscription_type'])
trip.head(3)

,duration,start_date,start_station_id,subscription_type
0,396,8/27/2015 8:36,50,Subscriber
1,636,7/28/2014 22:06,67,Subscriber
2,334,6/9/2014 8:42,77,Subscriber


In [5]:
weather = pd.read_csv('data/weather.csv')

In [18]:
def preprocess(station, weather, trip):
    station = preprocessStation(station)
    weather = preprocessWeather(weather)
    trip = preprocessTrip(trip)
    trip = trip.merge(station, left_on='start_station_id', right_on='id', inplace=True)
    trip = trip.merge(weather, left_on=['start_date', 'zip_code'], right_on=['date', 'zip_code'], how='left')
    return trip
    
def preprocessStation(df):
    df = df.copy()
    df["zip_code"] = df.city.apply(zipCode)
    df.drop(labels=['city'], axis=1, inplace=True)
    return df

def preprocessTrip(df):
    df = df.copy()
    df['start_date'] = df['start_date'].apply(lambda x : pd.to_datetime(x))
    df['time'] = df.start_date.apply(lambda x: 60 * x.hour + x.minute)
    df['year'] = df.start_date.apply(lambda x : x.year)
    df['month'] = df.start_date.apply(lambda x : x.month)
    df['day'] = df.start_date.apply(lambda x : x.day)
    df['dayofweek'] = df.start_date.apply(lambda x : x.isoweekday())
#    df = df.merge(holidays.to_frame(), left_on='start_date', right_on=holidays.to_frame().index, how='outer')
    return df

def preprocessWeather(df):
    df = df.copy()
    cleanPrecipitation(df)
    cleanEvents(df)
    df = df.fillna(0)
    df['date'] = df.date.apply(lambda x: pd.to_datetime(x).date())
    return df

def zipCode(city):
    if city == 'San Francisco' : return 94107
    elif city == 'San Jose' : return 95113
    elif city == 'Redwood City' : return 94063
    elif city == 'Palo Alto' : return 94301
    elif city == 'Mountain View' : return 94041

def cleanPrecipitation(df):
    df.dropna(subset=["precipitation_inches"], inplace=True)
    df['precipitation_inches'] = df.precipitation_inches.apply(lambda x:
                                                               0.005 if (x == 'T')
                                                               else float(x))
def cleanEvents(df):
    df['events'] = df.events.apply(lambda x: "Normal" if pd.isnull(x) else x)
    df['events'] = df.events.apply(lambda x: "Rain" if x == "rain" else x)

from pandas.tseries.holiday import USFederalHolidayCalendar as calendarUSFH
cal = calendarUSFH()
holidays = cal.holidays(return_name=True, start=pd.to_datetime('20130101'), end=pd.to_datetime('20151231'))

In [10]:
station.head(1)

,id,zip_code
0,2,95113


In [11]:
preprocessTrip(trip)
trip.head(1)

,duration,start_date,start_station_id,subscription_type,time,year,month,day,dayofweek
0,396,2015-08-27 08:36:00,50,Subscriber,516,2015,8,27,4


In [16]:
pd.merge(trip, station, left_on='start_station_id', right_on='id')

,duration,start_date,start_station_id,subscription_type,time,year,month,day,dayofweek,id,zip_code
0,396,2015-08-27 08:36:00,50,Subscriber,516,2015,8,27,4,50,94107
1,4120,2014-08-30 14:44:00,50,Customer,884,2014,8,30,6,50,94107
2,4675,2014-03-23 16:25:00,50,Customer,985,2014,3,23,7,50,94107
3,1595,2014-05-26 13:01:00,50,Customer,781,2014,5,26,1,50,94107
4,384,2013-12-23 08:48:00,50,Subscriber,528,2013,12,23,1,50,94107
5,1336,2014-12-07 12:06:00,50,Subscriber,726,2014,12,7,7,50,94107
6,1090,2015-08-26 08:07:00,50,Subscriber,487,2015,8,26,3,50,94107
7,488,2014-11-18 09:11:00,50,Subscriber,551,2014,11,18,2,50,94107
8,330,2014-12-16 08:08:00,50,Subscriber,488,2014,12,16,2,50,94107
9,732,2015-05-08 16:26:00,50,Subscriber,986,2015,5,8,5,50,94107


In [17]:
weather.head(1)

,date,max_temperature_f,mean_temperature_f,min_temperature_f,max_dew_point_f,mean_dew_point_f,min_dew_point_f,max_humidity,mean_humidity,min_humidity,...,mean_visibility_miles,min_visibility_miles,max_wind_Speed_mph,mean_wind_speed_mph,max_gust_speed_mph,precipitation_inches,cloud_cover,events,wind_dir_degrees,zip_code
0,8/29/2013,74.0,68.0,61.0,61.0,58.0,56.0,93.0,75.0,57.0,...,10.0,10.0,23.0,11.0,28.0,0,4.0,NaN,286.0,94107
